In [1]:
from facebook.driver_initialization import Initializer
from facebook.driver_utils import Utilities
from facebook.element_finder import Finder
import time
import pandas as pd
import json
import re


In [2]:
def save_file(dataset, file_name):
    try:
        print('text')
        dataset['dates'] = list(dataset['dates'])
        dataset['text'] = list(dataset['text'])
        df = pd.DataFrame.from_dict(dataset, orient='index')
        df = df.transpose()
        filename = f'facebook/data/{file_name}.csv'
        df.to_csv(filename, index=False)
    except Exception as e:
        print(e)
        print('could not save the dataset')

In [3]:
def remove_numbers_and_ext(text):
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove "ext"
    text = re.sub(r'\bext\b', '', text, flags=re.IGNORECASE)

    text = re.sub(r'\bsp\b', '', text, flags=re.IGNORECASE)

    text = re.sub(r'\bah\b', '', text, flags=re.IGNORECASE)

    return text


In [5]:
def write_set_to_json(data_set):
    with open('sections.json', 'w') as json_file:
        json.dump(list(data_set), json_file)

def read_set_from_json():
    with open('sections.json', 'r') as json_file:
        data = json.load(json_file)
        return set(data)

# sections_set = read_set_from_json()

In [12]:
def load_location_dataset(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

location_dataset = load_location_dataset('locations/location_dataset.json')
municipalities = list(location_dataset.keys())

selected_sub = set(
    [
     'Boksburg',
     'Alberton',
     'Tembisa',
     'Katlehong',
     'Kempton Park',
     'Germiston',
     'Sandton',
     'Randburg',
     'Soweto'
     
     ]
)

facebook_sub_areas = set()
for municipality in municipalities:
    for sub_area in location_dataset[municipality].keys():
        for section in location_dataset[municipality][sub_area]['sections']:
            if sub_area in selected_sub:
                facebook_sub_areas.add(section)
                facebook_sub_areas.add(sub_area)

In [7]:
search_terms =  ['crime', 'stolen', 'hijacking', 'robbery', 'murder', 'theft', 'assault',
                 'buglary', 'rape', 'cash in transit', 'cit', 'hijacking', 'violence',
                'protest', 'strike', 'cable', 'shoplifting', 'drugs', 'mall robbery', 
                'looting', 'truck', 'stolen phone', 'robbed']


 #todo" please add rosebank, brammfortein, makhulong, woodmead

# search_terms_bank_terms = ['fnb scam', 'nedbank scam', 'capitec scam',
#                          'absa scam', 'african bank scam', 'sars scam']

# search_terms_bank_terms = set(search_terms_bank_terms)

# facebook_sub_areas.update(search_terms_bank_terms)

In [9]:
driver = Initializer(
    browser_name='chrome', headless=False
).init()

# login (Retru)
driver = Utilities.login(driver, url='https://www.facebook.com/')

time.sleep(3)
# search posts


for search_term in search_terms:
    # if section not in search_terms_bank_terms:
    #     section = remove_numbers_and_ext(section).strip()
    #     place_ = f'{search_term} {section}'.lower()
    # else:
    place_ = search_term

    # if place_ in sections_set:
    #     continue

    if '/' in place_:
        place_ = place_.replace('/', ' ')

    print(place_)

    # sections_set.add(place_)

    driver = Finder.search_posts(place_, driver)

    # get all posts
    feed_data = Finder.get_all_posts(driver)

    # get_posts_dates
    dates = Finder.get_posts_dates(driver)

    # process_dates
    processed_dates = Finder.process_dates(dates)

    # get_posts_names
    posts_names = Finder.get_posts_names(feed_data)


    # get_posts_text
    posts_text = Finder.get_posts_text(feed_data)

    # data dictionary to store results
    data = {
        'dates': set(),
        'text': set()
    }

    data['dates'].update(set(processed_dates))
    data['text'].update(set(posts_text))

    retry = 4
    prev_data = None
    while True:
        time.sleep(2)
        Utilities.scroll_down(driver)
        Utilities.wait_until_completion(driver)
        Utilities.wait_until_posts_appear(driver)

        # get all posts
        feed_data = Finder.get_all_posts(driver)

        if prev_data == feed_data and retry < 0:
            save_file(dataset=data, file_name=f'_{search_term}')
            break
        if prev_data == feed_data:
            retry -= 1

        prev_data = feed_data

        # get_posts_dates
        dates = Finder.get_posts_dates(driver)

        # process_dates
        processed_dates = Finder.process_dates(dates)
        for date in processed_dates:
            if date not in data['dates']:
                data['dates'].add(date)

        # get_posts_text
        posts_text = Finder.get_posts_text(feed_data)
        for post in posts_text:
            if post  not in data['text']:
                data['text'].add(post)
                
    # write_set_to_json(sections_set)
    # time.sleep(5)

driver.close()
driver.quit()

[WDM] - Current google-chrome version is 115.0.5790
[WDM] - Get LATEST driver version for 115.0.5790


ValueError: There is no such driver by url http://chromedriver.storage.googleapis.com/LATEST_RELEASE_115.0.5790